In [1]:
import librosa
import librosa.display
import obspy
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py
import tsfel
import pandas as pd
import csv
import wave

In [2]:
# getting wav_files_paths
wav_files_paths = []

# open the CSV file in read mode
with open('wav_files_paths.csv', 'r') as file:
    reader = csv.reader(file)
    # Iterate through the rows in the CSV file
    for row in reader:
        # Append each row to the list
        wav_files_paths.append(row[0])

print(wav_files_paths[0])
print(wav_files_paths[30])

./data/wav/OS_7_05_2019_08_24_00_.wav
./data/wav/rpi-port-townsend_2019_11_14_14_28_00.wav


In [3]:
# converting annotations.tsv into df
annotations_file_path = './data/annotations.tsv'
df = pd.read_csv(annotations_file_path, sep='\t')
df.head()

,dataset,wav_filename,start_time_s,duration_s,location,date,pst_or_master_tape_identifier
0,podcast_round1,60012.wav,34.126,2.918,"Dabob Bay, Seattle, Washington",1960-10-28,60012
1,podcast_round1,60012.wav,36.816,2.588,"Dabob Bay, Seattle, Washington",1960-10-28,60012
2,podcast_round1,60012.wav,42.550,2.055,"Dabob Bay, Seattle, Washington",1960-10-28,60012
3,podcast_round1,60012.wav,44.606,2.410,"Dabob Bay, Seattle, Washington",1960-10-28,60012
4,podcast_round1,60012.wav,46.636,3.425,"Dabob Bay, Seattle, Washington",1960-10-28,60012


In [4]:
# find number of unique filenames in annotations.tsv
num_unique_filenames = df['wav_filename'].nunique()
print(num_unique_filenames)

2863


In [5]:
# find number of unique filenames in wav directory
!find ./data/wav -type f | wc -l

# since num wav files > num filenames in annotations.tsv, I will iterate through the
# ones in annotations.tsv

2976


In [6]:
# create orca_call_df which has cols wav_filename, start_time_s, end_time_s

# create new df with cols wav_filename, start_time_s as cols
orca_call_df = df[['wav_filename', 'start_time_s']]

# add end_time_s col by adding duration_s to start_time_s
orca_call_df['end_time_s'] = df['start_time_s'] + df['duration_s']

orca_call_df

/tmp/ipykernel_105031/2355072858.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orca_call_df['end_time_s'] = df['start_time_s'] + df['duration_s']


,wav_filename,start_time_s,end_time_s
0,60012.wav,34.126,37.044
1,60012.wav,36.816,39.404
2,60012.wav,42.550,44.605
3,60012.wav,44.606,47.016
4,60012.wav,46.636,50.061
...,...,...,...
5532,rpi-port-townsend_2020_09_08_14_55_00.wav,0.000,0.000
5533,rpi-port-townsend_2020_09_08_14_28_00.wav,0.000,0.000
5534,rpi-port-townsend_2020_10_08_02_25_00.wav,0.000,0.000
5535,rpi-port-townsend_2020_09_08_15_30_00.wav,0.000,0.000


In [7]:
# REMOVE DATA WHERE OVERLAPPING WHALE CALLS IN SAME AUDIO FILES
# functions to help removes all rows from df that have given filename
def remove_filenames(df, filenames_list):
  for filename in filenames_list:
    df = remove_filename(df, filename)
  return df

def remove_filename(df, filename):
  df = df.reset_index(drop=True)
  df = df[df['wav_filename'] != filename]
  return df

def find_rows_to_be_deleted(df):
    to_drop_filename = []
    for i in range(len(df) - 1):
        if (df.loc[i, 'wav_filename'] == df.loc[i + 1, 'wav_filename'] and 
            df.loc[i, 'end_time_s'] > df.loc[i + 1, 'start_time_s']):
          to_drop_filename.append(df.loc[i, 'wav_filename'])
    return to_drop_filename

In [8]:
to_drop_filename = find_rows_to_be_deleted(orca_call_df)
orca_call_df = remove_filenames(orca_call_df, to_drop_filename)
orca_call_df

,wav_filename,start_time_s,end_time_s
0,92303001.wav,0.0,1.059
1,92303002.wav,0.0,1.107
2,92303003.wav,0.0,1.306
3,92303004.wav,0.0,1.466
4,92303005.wav,0.0,2.026
...,...,...,...
2992,rpi-port-townsend_2020_09_08_14_55_00.wav,0.0,0.000
2993,rpi-port-townsend_2020_09_08_14_28_00.wav,0.0,0.000
2994,rpi-port-townsend_2020_10_08_02_25_00.wav,0.0,0.000
2995,rpi-port-townsend_2020_09_08_15_30_00.wav,0.0,0.000


In [9]:
# REMOVE DATA WHERE START_TIME_S = END_TIME_S = 0
orca_call_df = orca_call_df[~((orca_call_df['start_time_s'] == 0) & (orca_call_df['end_time_s'] == 0))]
orca_call_df

,wav_filename,start_time_s,end_time_s
0,92303001.wav,0.000000,1.059000
1,92303002.wav,0.000000,1.107000
2,92303003.wav,0.000000,1.306000
3,92303004.wav,0.000000,1.466000
4,92303005.wav,0.000000,2.026000
...,...,...,...
2820,rpi-bush-point_2020_09_27_23_30_00.wav,24.500000,26.950000
2821,rpi-bush-point_2020_09_27_23_30_00.wav,31.146750,32.826524
2822,rpi-bush-point_2020_09_27_23_30_00.wav,49.502321,51.450000
2823,rpi-bush-point_2020_09_27_23_30_00.wav,52.593964,54.093964


In [10]:
# saving clean orca_call_df to csv file
csv_file_path = './data/orca_data.csv'
orca_call_df.to_csv(csv_file_path, index=False)

In [11]:
# CREATING NOISE_DF
# create wav_filenames_len_dict which contains wav_filename and total_length
directory = './data/wav'

# initialize lists to store filenames and lengths
filenames = []
lengths = []

# iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        file_path = os.path.join(directory, filename)
        
        # load wav file
        y, sr = librosa.load(file_path, sr=None)
        # calculate length in seconds
        length = librosa.get_duration(y=y, sr=sr)
        
        # append the filename and length to the lists
        filenames.append(filename)
        lengths.append(length)

# creating dict where keys are wav_filenames and values are total_lengths
wav_filenames_len_dict = dict(zip(filenames, lengths))

In [12]:
print(len(wav_filenames_len_dict))

2975


In [13]:
noise_df = pd.DataFrame()

In [14]:
for wav_filename in wav_filenames_len_dict:
    if wav_filename in orca_call_df['wav_filename'].values:
        # create df with only rows of orca_call_df
        times_df = orca_call_df[orca_call_df['wav_filename'] == wav_filename][['start_time_s', 'end_time_s']]
        times_df = times_df.sort_values(by='start_time_s')
        times = []
        # fills times list with times in times_df
        for index, row in times_df.iterrows():
          times.append(row['start_time_s'])
          times.append(row['end_time_s'])
        # add 0 or length of audio file if necessary
        noise_start_index = 1
        # adding 0 if necessary:
        if times[0] != 0: # if whale call doesn't start at 0s
            times.insert(0, 0)
            noise_start_index = 0
        # adding length of audio file if necessary
        if times[len(times)-1] != wav_filenames_len_dict[wav_filename]: # if whale call doesn't end when the wav file ends
            length = wav_filenames_len_dict[wav_filename]
            times.insert(len(times), length)
        # populate noise_df
        if noise_start_index == 0:
            while len(times) > 1: # times is not empty
                new_row = {'wav_filename': wav_filename, 'start_time_s': times[0], 'end_time_s': times[1]}
                times.pop(0)
                times.pop(0)
                noise_df = pd.concat([noise_df, pd.DataFrame([new_row])], ignore_index=True)
        else: # noise_start_index = 1 eg: [0, 3.6, 4.0, 6.0, 7.7, 10]
            times.pop(0)
            while len(times) > 1: # times is not empty
                new_row = {'wav_filename': wav_filename, 'start_time_s': times[0], 'end_time_s': times[1]}
                times.pop(0)
                times.pop(0)
                noise_df = pd.concat([noise_df, pd.DataFrame([new_row])], ignore_index=True)

noise_df

,wav_filename,start_time_s,end_time_s
0,9778000Z.wav,1.4787,1.47875
1,9230001A.wav,4.0400,4.04030
2,97754016.wav,1.2858,1.28380
3,9230001J.wav,3.9100,3.91070
4,9230001L.wav,3.8630,3.86345
...,...,...,...
2282,9230400J.wav,1.9550,1.95510
2283,9775901N.wav,2.4957,2.49575
2284,9769201K.wav,3.2240,3.22435
2285,97692003.wav,3.2100,3.21060


In [15]:
# saving clean noise_df to csv file
csv_file_path = './data/noise_data.csv'
noise_df.to_csv(csv_file_path, index=False)